In [51]:
# Agent model library
import agentpy as ap
# Visualization
import matplotlib.pyplot as plt
import IPython
# Random
from random import randint

In [52]:
class junctionModel(ap.Model):
    def setup(self):
        # Minimum amount of cars
        if self.p.cars < 5: self.p.cars = 5

        north_positions = set()
        west_positions = set()
        east_positions = set()
        south_positions = set()

        for car in range(self.p.cars):
            # north-bound
            if car % 4 == 0:
                new_position = (4, randint(6, 8))
                while new_position in north_positions:
                    new_position = (4, randint(6, 8))
                north_positions.add(new_position)
            # west-bound
            elif car % 4 == 1:
                new_position =  (randint(6, 8), 4)
                while new_position in west_positions:
                    new_position = (randint(6, 8), 4)
                west_positions.add(new_position)
            # south-bound
            elif car % 4 == 2:
                new_position =  (4, randint(0, 2))
                while new_position in south_positions:
                    new_position = (4, randint(0, 2))
                south_positions.add(new_position)
            # east-bound
            elif car % 4 == 3:
                new_position =  (randint(0, 2), 4)
                while new_position in east_positions:
                    new_position = (randint(0, 2), 4)
                east_positions.add(new_position)

        grass_positions = [(4, 4)]
        # upper left quarter
        for i in range(3): # x axis iteration
            for j in range(4): # y axis iteration
                grass_positions.append((i, j)) # Filling the array of the grass agent positions
        # lower left quarter
        for i in range(5, 9):
            for j in range(3):
                grass_positions.append((i,j))
        # upper right quarter
        for i in range(4):
            for j in range(6, 9):
                grass_positions.append((i,j))
        # lower right quarter
        for i in range(6, 9):
            for j in range(5, 9):
                grass_positions.append((i,j))

        intersection_positions = [
            (3, 3), (4, 3), (5, 3),
            (3, 4),         (5, 4),
            (3, 5), (4, 5), (5, 5),
        ]

        # Create agents (cars and sidewalks)
        n_grass = int((3*4)*4)
        grass = ap.AgentList(self, n_grass)
        cars_n = ap.AgentList(self, len(north_positions))
        cars_w = ap.AgentList(self, len(west_positions))
        cars_s = ap.AgentList(self, len(south_positions))
        cars_e = ap.AgentList(self, len(east_positions))
        intersection_agents = ap.AgentList(self, 8)

        grass.status = 0
        cars_n.status = 1
        cars_w.status = 6
        cars_s.status = 11
        cars_e.status = 16
        for status, agent in zip(range(21, 29), intersection_agents):
            agent.status = status

        self.agents = grass
        self.agents.extend(cars_n)
        self.agents.extend(cars_w)
        self.agents.extend(cars_s)
        self.agents.extend(cars_e)
        self.agents.extend(intersection_agents)

        self.ground = ap.Grid(self, (self.p.width, self.p.height), track_empty = True, check_border = True)
        self.ground.add_agents(grass, grass_positions, empty = False)
        self.ground.add_agents(cars_n, north_positions, empty = False)
        self.ground.add_agents(cars_w, west_positions, empty = False)
        self.ground.add_agents(cars_s, south_positions, empty = False)
        self.ground.add_agents(cars_e, east_positions, empty = False)
        self.ground.add_agents(intersection_agents, intersection_positions, empty = False)
        
    def step(self):
        
        moving_cars = self.agents
        #          x   y
        west  = ( -1,  0 )
        east  = (  1,  0 )
        north = (  0, -1 )
        south = (  0,  1 )

        for car in moving_cars:
            if (car.status == 1):
                self.ground.move_by(car, north)
            if (car.status == 6):
                self.ground.move_by(car, west)
            if (car.status == 11):
                self.ground.move_by(car, south)
            if( car.status == 16):
                self.ground.move_by(car, east)

    def end(self):
        self.report('Simulacion terminada')


In [53]:
parameters = {
# Simulation steps
'cars': 10,
'width': 9,
'height': 9,
'steps': 100,
}

In [54]:
def animation_plot(model, ax):
    attr_grid = model.ground.attr_grid('status')
    color_dict = {
        0: '#aacd00',  # grass
        1: '#ff0000',  2: '#ff0000',  3: '#ff0000',  4: '#ff0000',  5: '#ff0000',  # north-bound
        6: '#32a852',  7: '#32a852',  8: '#32a852',  9: '#32a852',  10: '#32a852', # west-bound
        11: '#329ea8', 12: '#329ea8', 13: '#329ea8', 14: '#329ea8', 15: '#329ea8', # south-bound
        16: '#8332a8', 17: '#8332a8', 18: '#8332a8', 19: '#8332a8', 20: '#8332a8', # east-bound
        21: '#ffffff', 22: '#ffffff', 23: '#ffffff', 24: '#ffffff', # roundabout cells
        25: '#ffffff', 26: '#ffffff', 27: '#ffffff', 28: '#ffffff',
        None: '#ffffff'
    }
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of a junction")

In [55]:
fig, ax = plt.subplots()
model = junctionModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))